# INM706: Inference (Seq2Seq)

This is notebook is for inference of the models found in the [City-INM706](https://github.com/yasirbarlas/City-INM706) Github repository.

The model used for inference here is our improved/final model, and so the parameters reflect this.

### Import Libraries and Models

In [1]:
# Import relevant libraries and models

from models import *
from dataset import *

import random
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.nist_score import sentence_nist

### Create Model From Checkpoint

In [2]:
# Set device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

# Get model checkpoint
model = torch.load(f"checkpoints/checkpoint_seq2seq_latest.pth", map_location = device)

# Encoder and Decoder, replace 128 with your hidden dimension size and other parameters
# The parameters given are for our improved/final model
encoder = EncoderRNN(model["input_lang"].n_words, 128, bidirectional = True, num_layers = 2, layer_norm = True, dropout_p = 0.1).to(device)
decoder = DecoderRNN(128, model["output_lang"].n_words, num_layers = 2, use_lstm = True, max_seq_len = 50).to(device)

# Split into Encoder and Decoder
encoder.load_state_dict(model["encoder_state_dict"])
decoder.load_state_dict(model["decoder_state_dict"])

# Ensure the Encoder and Decoder are in evaluation mode
encoder.eval()
decoder.eval()

DecoderRNN(
  (embedding): Embedding(96093, 128)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True)
  (out): Linear(in_features=128, out_features=96093, bias=True)
)

### Test Dataset

In [3]:
# Make test dataset (change max_seq_len to your maximum sentence word length)
# Choose any dataset by inserting their paths as below (or skip if not interested in model testing)
test_dataset = TranslationDataset(lang1 = "test2008-en.txt", lang2 = "test2008-fr.txt", max_seq_len = 50, reverse = False, directory = "../fr-en/")

Read 2000 sentence pairs
Trimmed to 1766 sentence pairs
Counting words...
Counted words:
test2008-en.txt 5003
test2008-fr.txt 5924


### Functions for Inference

These functions are necessary to display sentences in human speech, rather than in tokens.

In [4]:
# Functions for inference

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(" ") if word in lang.word2index]

def tensorFromSentence(lang, sentence):
    EOS_token = 1
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long, device = device).view(1, -1)

def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    EOS_token = 1
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, None

def evaluateDataset(encoder, decoder, dataset, model):
    bleus = []
    nists = []
    skipped = 0

    for pair in dataset.pairs:
        # Check if all words in the input sentence exist in the input_lang word2index
        input_valid = all(word in model["input_lang"].word2index for word in pair[0].split())

        if not input_valid:
            skipped += 1
            #print(f"Skipped sentence due to unknown words: {pair[0]} -> {pair[1]}")
            continue

        print(">", pair[0])
        print("=", pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], model["input_lang"], model["output_lang"])
        output_sentence = " ".join(output_words)
        print("<", output_sentence)

        # BLEU
        ref = [pair[1].split()]
        bleu = sentence_bleu(ref, output_sentence.split()[:-1], smoothing_function = SmoothingFunction().method1)
        nist = sentence_nist(ref, output_sentence.split()[:-1], min(4, len(output_sentence.split()[:-1])))
        bleus.append(bleu)
        nists.append(nist)
        print("BLEU Score", bleu)
        print("NIST Score", nist)
        print("")

    if bleus:
        print("Average BLEU Score:", np.mean(bleus))
    if nists:
        print("Average NIST Score:", np.mean(nists))
    print(f"Total skipped sentences: {skipped}")

### Test Set Evaluation

Calculate BLEU and NIST for the test set imported earlier.

In [5]:
# Test Dataset Results
evaluateDataset(encoder, decoder, test_dataset, model)

> these are as we know very tricky negotiations and raise fears of a setback or a watered down agreement in nice which as you have already acknowledged mr moscovici would be even more serious
= ces negociations nous le savons sont tres difficiles et font craindre un echec ou un accord a minima a nice ce qui serait encore plus grave vous en avez deja convenu monsieur le ministre
< nous savons que nous avons des negociations et des negociations tres difficiles comme un exemple de partialite et de desaccord avec l accord de nice qui est encore plus important que l accord de nice <EOS>
BLEU Score 0.020904996083879822
NIST Score 1.7157015455715454

> any other procedure would mean a huge democratic deficit
= toute autre facon de faire entrainerait un deficit democratique majeur
< un autre rapport pourrait etre un peu transparent <EOS>
BLEU Score 0.025725069574826766
NIST Score 0.6732186484917868

> mr president mr president of the commission mr president in office ladies and gentlemen i sho

### Single Sentence Evaluation

One may wish to enter their own sentence (one that contains the words used during training) for inference. You should use lowercase without any punctuation.

In [6]:
sentence = "mr president ladies and gentlemen in his policy statement yesterday mr prodi the president of the commission said that whoever weakened any institution of the european union weakened the union as a whole"

decoder_output, _ = evaluate(encoder, decoder, sentence, model["input_lang"], model["output_lang"])
output_sentence = " ".join(decoder_output)
print(output_sentence)

monsieur le president mesdames et messieurs hier a m prodi la commission prodi a declare a l unanimite la propagande politique de l union europeenne qui a ete utilise par l union europeenne qui a perdu la valeur de son propre institution <EOS>
